In [1]:
import wrds, os
import numpy as np
import pandas as pd

In [2]:
os.chdir('..')

---

In [3]:
db = wrds.Connection()

Enter your WRDS username [maltemax]:mmx900
Enter your password:········
WRDS recommends setting up a .pgpass file.
Create .pgpass file now [y/n]?: n
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


In [4]:
df = db.get_table(library = 'crsp', table = 'comphist')

In [5]:
df = df[['gvkey', 'hchgdt', 'hchgenddt', 'hfic', 'hsic', 'hcik']]

In [6]:
df.head()

,gvkey,hchgdt,hchgenddt,hfic,hsic,hcik
0,001000,2007-04-14,2023-08-11,USA,3089.0,None
1,001000,2023-08-12,None,USA,3089.0,None
2,001001,2007-04-14,2011-02-11,USA,5812.0,None
3,001001,2011-02-12,2012-03-09,USA,5812.0,0000723576
4,001001,2012-03-10,None,USA,5812.0,0000723576


In [8]:
df.loc[df['hcik'] == '0000001750']

,gvkey,hchgdt,hchgenddt,hfic,hsic,hcik
18,001004,2007-04-14,2007-09-07,USA,5080.0,0000001750
21,001004,2007-09-08,2008-08-08,USA,5080.0,0000001750
22,001004,2008-08-09,2009-07-31,USA,5080.0,0000001750
23,001004,2009-08-01,2010-06-11,USA,5080.0,0000001750
24,001004,2010-06-12,2010-08-13,USA,5080.0,0000001750
25,001004,2010-08-14,2011-10-07,USA,5080.0,0000001750
26,001004,2011-10-08,2012-08-10,USA,5080.0,0000001750
27,001004,2012-08-11,2013-08-09,USA,5080.0,0000001750
28,001004,2013-08-10,2014-08-08,USA,5080.0,0000001750
29,001004,2014-08-09,2015-08-07,USA,5080.0,0000001750


In [7]:
df['date_from'], df['date_to'] = pd.to_datetime(df['hchgdt']), pd.to_datetime(df['hchgenddt'])

df['fyear_from'] = np.where(df['date_from'].dt.month < 6,
                            df['date_from'].dt.year,
                            df['date_from'].dt.year + 1)

df['fyear_to'] = np.where(df['date_to'].dt.month < 6,
                          df['date_to'].dt.year - 1,
                          df['date_to'].dt.year)

In [8]:
df = df.loc[~(df['fyear_from'] > df['fyear_to'])].reset_index(drop = True)

In [9]:
df['fyear_to'] = df['fyear_to'].fillna(2024)

In [10]:
df = df.drop(columns = ['hchgdt', 'hchgenddt', 'date_from', 'date_to'])

In [11]:
between = df.to_dict(orient = 'index')

In [12]:
in_between = []

for k, v in between.items():
    in_between.append([i for i in range(int(v['fyear_from']), int(v['fyear_to']) + 1)])

In [13]:
df['fyear'] = pd.Series(in_between)

In [14]:
df = df.explode('fyear')

In [15]:
df = df.drop(columns = ['fyear_from', 'fyear_to'])

In [17]:
df.to_csv(os.path.join(os.getcwd(), '3_pipeline', '1_intermediate', 'comphist_prepared.tsv'),
          sep = '\t', index = False)